In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        pass
        #print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from sklearn.preprocessing import MultiLabelBinarizer
import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.python.framework import ops
import math
import glob
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [ ]:
train_image_path = '../input/plant-pathology-2021-fgvc8/train_images'
test_image_path = '../input/plant-pathology-2021-fgvc8/test_images'
train_df_path = '../input/plant-pathology-2021-fgvc8/train.csv'
test_df_path = '../input/plant-pathology-2021-fgvc8/sample_submission.csv'

In [ ]:
train_image_path = '../input/resized-plant2021/img_sz_512'

In [ ]:
df_train = pd.read_csv(train_df_path)
df_train.head()

In [ ]:
df_train["labels"]=df_train["labels"].apply(lambda x:x.split(" "))
df_train.head()

In [ ]:
IMG_SIZE = 512
BATCH_SIZE = 128
DATASET_SIZE = 18632
train_size = int(0.9 * DATASET_SIZE)
#16,768
val_size = int(0.1 * DATASET_SIZE)

datagen=tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

train_generator=datagen.flow_from_dataframe(
    dataframe=df_train[:train_size],
    directory=train_image_path,
    x_col="image",
    y_col="labels",
    batch_size=BATCH_SIZE,
    seed=42,
    shuffle=False,
    class_mode="categorical",
    classes=['complex', 'frog_eye_leaf_spot', 'healthy', 'powdery_mildew', 'rust', 'scab'],
    target_size=(IMG_SIZE, IMG_SIZE)
)

valid_generator=datagen.flow_from_dataframe(
    dataframe=df_train[train_size:],
    directory = train_image_path,
    x_col="image",
    y_col="labels",
    batch_size=BATCH_SIZE,
    seed=42,
    shuffle=False,
    class_mode="categorical",
    classes=['complex', 'frog_eye_leaf_spot', 'healthy', 'powdery_mildew', 'rust', 'scab'],
    target_size=(IMG_SIZE,IMG_SIZE)
)

In [ ]:
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)


#base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')
base_model = tf.keras.models.load_model('../input/pretrainedmodel/MobileNet_512.h5')
base_model.trainable = False

In [ ]:

model = tf.keras.Sequential([
      base_model,
      layers.GlobalAveragePooling2D(),
      layers.Dense(6, activation='sigmoid',activity_regularizer=tf.keras.regularizers.l2(0.01))
    ])

model.compile(loss='binary_crossentropy', optimizer='adam')

model.summary()


history = model.fit(train_generator, epochs=15, validation_data = valid_generator)

model.save('saved_model/model1')

In [ ]:
test_list = os.listdir(test_image_path)
col_names = ['image']
test_df = pd.DataFrame(test_list, columns=col_names)

test_datagen=tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
test_generator=test_datagen.flow_from_dataframe(
dataframe=test_df,
directory=test_image_path,
x_col="image",
batch_size=1,
seed=42,
shuffle=False,
class_mode=None,
target_size=(IMG_SIZE,IMG_SIZE)
)

test_generator.reset()
pred=model.predict(test_generator, verbose=1)

pred_bool = (pred >0.5)
predictions=[]
labels = train_generator.class_indices
labels = dict((v,k) for k,v in labels.items())
for row in pred_bool:
    l=[]
    for index,cls in enumerate(row):
        if cls:
            l.append(labels[index])
    predictions.append(" ".join(l))
filenames=test_generator.filenames
results=pd.DataFrame({"image":filenames,
                      "labels":predictions})

results = results.fillna("healthy")
results.head()

results.to_csv("submission.csv",index=False)